# Imports

In [ ]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# Read CSV data

In [ ]:
df2 = pd.read_csv('toddlerData.csv',na_values='?')
df2.head()

# Get dummies

In [ ]:
within24_36= pd.get_dummies(df2['Age_Mons']>24,drop_first=True)
within0_12 = pd.get_dummies(df2['Age_Mons']<13,drop_first=True)
male=pd.get_dummies(df2['Sex'],drop_first=True)
ethnics=pd.get_dummies(df2['Ethnicity'],drop_first=True)
jaundice=pd.get_dummies(df2['Jaundice'],drop_first=True)
ASD_genes=pd.get_dummies(df2['Family_mem_with_ASD'],drop_first=True)
ASD_traits=pd.get_dummies(df2['Class/ASD Traits '],drop_first=True)

# Generate Columns

In [ ]:
final_data= pd.concat([within0_12,within24_36,male,ethnics,jaundice,ASD_genes,ASD_traits],axis=1)
final_data.columns=['within0_12','within24_36','male','Latino','Native Indian','Others','Pacifica','White European','asian','black','middle eastern','mixed','south asian','jaundice','ASD_genes','ASD_traits']
final_data.head()

# Data prep
Prepare the data and split data into testing and training

In [ ]:
X= final_data.iloc[:,:-1]
y= final_data.iloc[:,-1]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.9,random_state=101)

# Models
Setup 3 Classification models to be ensembled later on

In [ ]:
lgr=LogisticRegression()
lgr.fit(X_train,y_train)

rfc= RandomForestClassifier(n_estimators=500)
rfc.fit(X_train,y_train)

knn= KNeighborsClassifier(n_neighbors=105)
knn.fit(X_train,y_train)

# Results
Define estimators, setup ensemble system, train the ensemble system, and produce a confusion matrix and classificaiton report

In [ ]:
estimators=[('knn', knn), ('rfc', rfc), ('log_reg', lgr)]
ensemble = VotingClassifier(estimators, voting='hard')
ensemble.fit(X_train, y_train)
ensemblePred=ensemble.predict(X_test)

print(confusion_matrix(y_test,ensemblePred))
print(classification_report(y_test,ensemblePred))